# Import train, validation, and test datasets

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from keras.preprocessing.image import ImageDataGenerator

train_normalize = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_normalize = ImageDataGenerator(rescale=1./255)
val_normalize = ImageDataGenerator(rescale=1./255)

main_dir = 'splits/split1/'
train_set = train_normalize.flow_from_directory(directory=main_dir+"train",target_size=(225,225),class_mode='categorical')
val_set = val_normalize.flow_from_directory(directory=main_dir+"val",target_size=(225,225),class_mode="categorical")
test_set = test_normalize.flow_from_directory(directory=main_dir+"test",target_size=(225,225),class_mode="categorical")

# Create, train, and validate the model

In [8]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tensorflow.keras import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Flatten,Dense,MaxPooling2D,Conv2D,Dropout
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam
import os
import random
import time
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# !pip install tensorflow-addons
# import tensorflow_addons as tfa



# ########## Good architecture

learning_rate = 0.001
epochs = 15
batch_size = 16

model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(fig_size, fig_size, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

opt = Adam(learning_rate=learning_rate)
checkpoint_path = "gdfit_1_20e_16b-{epoch:1d}.h5"
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,  # Save only the weights
    save_freq='epoch' # Save for every epoch
)


# ########## Overfit architecture
# learning_rate = 0.001
# epochs = 30
# batch_size = 16
# model = Sequential()
# model.add(Conv2D(64, (3, 3), input_shape=(fig_size, fig_size, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu'))  # Additional convolutional layer
# model.add(MaxPooling2D((2, 2)))
# model.add(Flatten())
# model.add(Dense(512, activation='relu'))  # Increased neurons
# model.add(Dense(256, activation='relu'))  # Increased neurons
# model.add(Dense(128, activation='relu'))  # Increased neurons
# model.add(Dense(64, activation='relu'))  # Increased neurons
# model.add(Dense(32, activation='relu'))  # Increased neurons
# model.add(Dense(3, activation='softmax'))
# opt = Adam(learning_rate=learning_rate)
# checkpoint_path = "ovfit_1_30e_16b-{epoch:1d}.h5"
# checkpoint_callback = ModelCheckpoint(
#     filepath=checkpoint_path,
#     save_freq='epoch' # Save for every epoch
# )

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with adjusted parameters
history = model.fit(train_set, epochs=epochs, batch_size=batch_size, validation_data=val_set, callbacks=[checkpoint_callback])
# history = model.fit(train_set, epochs=epochs, batch_size=batch_size, validation_data=val_set)


# Plot architecture of created model

In [ ]:
plot_model(model,show_layer_names=True,show_shapes=True)

# Plot Training and Validation Accuracy and Loss

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Custom tick formatter function
def format_tick(x, pos):
    return int(x + 1)
# Plot training accuracy and loss
plt.figure(figsize=(12, 6), constrained_layout=True)
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy', color='red')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Training and Validation Accuracy')
plt.grid(True)  # Add grid
plt.xticks(range(0, epochs+1))  # Set x-axis ticks to every integer value
plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(format_tick))  # Apply custom tick formatter
plt.tight_layout()  # Adjust layout
plt.legend()

# Plot validation accuracy and loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss', color='red')
plt.plot(history.history['val_loss'], label='Validation Loss', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)  # Add grid
plt.xticks(range(0, epochs+1))  # Set x-axis ticks to every integer value
plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(format_tick))  # Apply custom tick formatterplt.tight_layout()  # Adjust layout
plt.show()
plt.tight_layout(pad=5.0)
plt.subplots_adjust(left=0.1, bottom=0.1,                    top=0.9, wspace=0.4,hspace=0.4)

# Metrics - using TP, FP, TN, FN - not tested

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix

def perf_measure(y_actual, y_pred):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_pred)):
        if y_actual[i]==y_pred[i]==1:
           TP += 1
        if y_pred[i]==1 and y_actual[i]!=y_pred[i]:
           FP += 1
        if y_actual[i]==y_pred[i]==0:
           TN += 1
        if y_pred[i]==0 and y_actual[i]!=y_pred[i]:
           FN += 1

    return(TP, FP, TN, FN)

def calculate_accuracy(tp, fp, tn, fn):
    return (tp + tn) / (tp + fp + tn + fn)

def calculate_recall(tp, fp, tn, fn):
    if (tp+fn) == 0:
        return 0
    else:
      return tp / (tp + fn)

def calculate_precision(tp, fp, tn, fn):
    if (tp+fp) == 0:
        return 0
    else:
        return tp / (tp + fp)

def calculate_f1_score(tp, fp, tn, fn):
    precision = calculate_precision(tp, fp, tn, fn)
    recall = calculate_recall(tp, fp, tn, fn)
    denominator = precision + recall
    if denominator == 0:
      return 0
    else:
      return 2 * (precision * recall) / (precision + recall)

# Lists to store metrics for each epoch
val_accuracy_list, val_recall_list, val_precision_list, val_f1_list = [], [], [], []

i_begin = 1
i_end = epochs + 1
model_name = "gdfit_1_20e_16b"
# Load each saved model and calculate evaluation metrics
for epoch in range(i_begin, i_end):  # Assuming you trained for 10 epochs
    print("epoch:", epoch)
    # Load model from saved checkpoint
    model.load_weights(model_name + f"-{epoch}.h5")

    # Make predictions on validation dataset
    val_predictions = model.predict(val_set)

    # Calculate confusion matrix
    true_labels = val_set.classes
    predicted_labels = np.argmax(val_predictions, axis=1)

    # Extract true positives, false positives, true negatives, and false negatives
    tp, fp, tn, fn = perf_measure(true_labels, predicted_labels)
    print("tp:", tp, " fp:", fp, "tn:", tn, "fn:", fn)

    # Calculate metrics based on TP, FP, TN, FN
    val_accuracy = calculate_accuracy(tp, fp, tn, fn)
    print("val_accuracy:", val_accuracy)
    val_recall = calculate_recall(tp, fp, tn, fn)
    print("val_recall:", val_recall)
    val_precision = calculate_precision(tp, fp, tn, fn)
    print("val_precision:", val_precision)
    val_f1 = calculate_f1_score(tp, fp, tn, fn)
    print("val_f1:", val_f1)

    # Append metrics to lists
    val_accuracy_list.append(val_accuracy)
    val_recall_list.append(val_recall)
    val_precision_list.append(val_precision)
    val_f1_list.append(val_f1)

# Metrics using scikit-learn

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

# Lists to store metrics for each epoch
val_accuracy_list, val_recall_list, val_precision_list, val_f1_list = [], [], [], []

i_begin = 1
i_end = epochs + 1
model_name = "gdfit_1_20e_16b"

# Load each saved model and calculate evaluation metrics
for epoch in range(i_begin, i_end):  # Assuming you trained for 10 epochs
    print("epoch:", epoch)
    # Load model from saved checkpoint
    model.load_weights(model_name + f"-{epoch}.h5")

    # Make predictions on validation dataset
    val_predictions = model.predict(val_set)

    # print(val_predictions)

    # Convert predictions to class labels
    predicted_labels = np.argmax(val_predictions, axis=1)

    # print(predicted_labels)

    # print(val_set.classes)
    # print(predicted_labels)

    # Calculate metrics based on confusion matrix
    val_accuracy = accuracy_score(val_set.classes, predicted_labels)
    val_recall = recall_score(val_set.classes, predicted_labels, average='macro')
    val_precision = precision_score(val_set.classes, predicted_labels, average='macro')
    val_f1 = f1_score(val_set.classes, predicted_labels, average='macro')

    # Append metrics to lists
    val_accuracy_list.append(val_accuracy)
    val_recall_list.append(val_recall)
    val_precision_list.append(val_precision)
    val_f1_list.append(val_f1)

    print("val_accuracy:", val_accuracy)
    print("val_recall:", val_recall)
    print("val_precision:", val_precision)
    print("val_f1:", val_f1)

# Plot metrics for each epoch

In [ ]:
# Plot metrics for each epoch
plt.figure(figsize=(8, 6))  # Adjust figure size if needed

colors = ["orange", "green", "purple", "blue"]

# Plot accuracy for each epoch
plt.plot(range(i_begin, i_end), val_accuracy_list, color=colors[0], label="Accuracy")
plt.plot(range(i_begin, i_end), val_recall_list, color=colors[1], label="Recall")
plt.plot(range(i_begin, i_end), val_precision_list, color=colors[2], label="Precision")
plt.plot(range(i_begin, i_end), val_f1_list, color=colors[3], label="F1 Score")

# Add labels and legend
plt.xlabel('Epoch')
plt.ylabel('Metric value')
plt.title('Validation Metrics over Epochs')
plt.legend()
plt.grid(True)  # Add grid
plt.xticks(range(i_begin, i_end))  # Set x-axis ticks to every integer value
plt.tight_layout()  # Adjust layout
plt.show()

# Plot Confusion Matrix for every 5th epoch

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Define a function to plot confusion matrix
def plot_confusion_matrix(matrix, title, ax, class_names):
    sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names, ax=ax)
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title(title, fontsize=10)


# Get the class names from the generator
class_names = list(val_set.class_indices.keys())

for epoch in range(i_begin, i_end):
    if epoch % 5 == 0:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))  # Adjust the width of the figure
        # fig.subplots_adjust(wspace=0.5)  # Adjust the horizontal space between subplots

        # Load the model weights from the checkpoint file for this epoch
        model.load_weights(model_name + f"-{epoch}.h5")

        # Get the predictions for the training and validation datasets
        train_predictions = model.predict(train_set)
        val_predictions = model.predict(val_set)

        # Get the true labels for the training and validation datasets
        train_true_labels = train_set.classes
        val_true_labels = val_set.classes

        # Calculate accuracy
        train_accuracy = accuracy_score(train_true_labels, np.argmax(train_predictions, axis=1))
        val_accuracy = accuracy_score(val_true_labels, np.argmax(val_predictions, axis=1))
        print("train_accuracy:", train_accuracy)
        print("val_accuracy:", val_accuracy)

        # Compute confusion matrix for training and validation datasets
        train_conf_matrix = confusion_matrix(train_true_labels, np.argmax(train_predictions, axis=1))
        val_conf_matrix = confusion_matrix(val_true_labels, np.argmax(val_predictions, axis=1))

        # Plot confusion matrices side by side
        fig.suptitle(f'Confusion Matrices (Epoch {epoch})')
        plot_confusion_matrix(train_conf_matrix, f'Training Confusion Matrix (Epoch {epoch})', ax1, class_names)
        plot_confusion_matrix(val_conf_matrix, f'Validation Confusion Matrix (Epoch {epoch})', ax2, class_names)
        plt.show()
